# **Build Model**
https://github.com/theanhtran372000/Vietnamese-News-Classification/blob/main/4.PhoBERT.ipynb



## **Install Library**

In [ ]:
pip install pyvi

In [ ]:
!pip3 install fairseq

In [ ]:
!pip3 install fastbpe

In [ ]:
!pip3 install transformers

In [ ]:
!pip install underthesea

In [ ]:
pip install -U flask-cors

In [ ]:
!pip install pytorch-model-summary

## **Import Library**

In [ ]:
import sys
import os
import numpy as np
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
import time
import random
import seaborn as sns
from tqdm import tqdm
from tqdm.notebook import tqdm
import argparse

from flask import Flask, request,render_template,jsonify
from flask_cors import CORS
import flask

from pyvi import ViTokenizer
from wordcloud import WordCloud, STOPWORDS

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch.nn import functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR
from torch.backends import cudnn
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, f1_score, classification_report, accuracy_score
from sklearn.utils import class_weight, resample, shuffle
from sklearn.manifold import TSNE

from collections import Counter
from imblearn.over_sampling import SMOTE
from underthesea import word_tokenize
import pickle

from fairseq.models.roberta import RobertaModel
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from transformers.modeling_utils import *
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
from transformers import AutoModel, AutoTokenizer

import argparse

app = Flask(__name__)
CORS(app)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Load Data***

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Research - CSR_NLP/full_df.csv', usecols=lambda x: x != 'Unnamed: 0')

In [ ]:
original_df = df.copy()
# Filter the dataframe to select rows with 'No CSR' label
no_csr_df = original_df[original_df['label'] == 'No CSR']

# Calculate the maximum number of 'No CSR' rows allowed
max_no_csr_rows = len(original_df) * 0.1

# Select a random sample of 'No CSR' rows up to the maximum allowed
no_csr_sample = no_csr_df.sample(n=int(max_no_csr_rows))

# Filter the original dataframe to exclude the selected 'No CSR' rows
original_df_filtered = original_df[original_df['label'] != 'No CSR']

# Randomly select remaining rows to achieve 50% of the original dataframe
random_sample = original_df_filtered.sample(frac=0.95)

# Concatenate the randomly selected rows with the 'No CSR' sample
new_df = pd.concat([random_sample, no_csr_sample])

# Shuffle the rows in the new dataframe
new_df = new_df.sample(frac=1)

# Reset the index of the new dataframe
new_df.reset_index(drop=True, inplace=True)


In [ ]:
new_df#.label.value_counts()

,name,year,text,label
0,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2016,Sacombank đã ban hành bộ tiêu chuẩn năng lực q...,CSR0_8
1,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2020,Sacombank cũng hướng đến sử dụng các ng...,CSR2_6
2,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2020,Hệ thống xử lý nước thải của SeABank đáp ứng t...,CSR2_1
3,30. NGÂN HÀNG TMCP ĐẠI CHÚNG VIỆT NAM,2013,"Tình thương có sức lan tỏa tự nhiên, vì vậy, k...",CSR5_0
4,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2017,"Hiện nay, Bộ chỉ tiêu KPIs đã phát huy tốt va...",CSR0_10
...,...,...,...,...
3325,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,"Trên thực tế, Thành viên Hội đồng Quản trị và ...",CSR0_1
3326,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,"Nhân dịp này, Chủ tịch nước Trương Tấn Sang đã...",CSR5_3
3327,2. NGÂN HÀNG TMCP KỸ THƯƠNG VIỆT NAM,2021,"Đồng thời, ông có kinh nghiệm tham gia quản ...",No CSR
3328,27. NGÂN HÀNG TMCP CÔNG THƯƠNG VIỆT NAM,2016,"Ngoài ra, VietinBank đã đầu tư hỗ trợ xây dựn...",CSR5_4


In [ ]:
test_df = new_df.copy()

# Preprocessing

In [ ]:
'''
Xóa HTML code
'''
def remove_html(text):
    return re.sub(r'<[^>]*>', '', text)

In [ ]:
'''
Chuẩn hóa bảng mã Unicode
  * Đưa hết những câu trong dataset về thành kiểu Unicode dựng sẵn.
'''
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

# Hàm tạo dict để convert định dạng cũ sang định dạng mới
def loaddicchar():
  dic = {}
  char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')
  charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')
  for i in range(len(char1252)):
      dic[char1252[i]] = charutf8[i]
  return dic

dicchar = loaddicchar()

# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [ ]:
'''
Chuẩn hóa kiểu gõ dấu tiếng Việt & đưa về dạng thường
  - Lỗi gõ dấu thường xảy ra do cách đánh máy của người viết báo
  - Chuyển các từ về dạng viết thường sẽ giảm số lượng đặc trưng mà mô hình phải học => tăng độ chính xác model
'''
# Danh sách nguyên âm với các dấu tương ứng
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

# Bảng chứa ký tự dấu ứng: dấu trắng, dấu huyền, sắc, hỏi, ngã, nặng ứng với các cột bên trên
bang_ky_tu_dau =  [ '', 'f', 's', 'r', 'x', 'j']

# dict để chuyển ký tự unicode sang chỉ số (i, j) trong đó i đặc trưng cho nguyên âm nào còn j đặc trưng cho dấu
nguyen_am_to_ids = {}

# Khởi tạo dict
for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

# Kiểm tra tính hợp lệ của một từ tiếng Việt đã được gõ đúng hay chưa dựa trên tính chất là:
# Các nguyên âm trong tiếng việt sẽ đứng cạnh nhau tạo thành cụm và mỗi từ chỉ có tối đa 1 cụm nguyên âm
# Ví dụ: Nguyễn -> Ng(uyễ)n, Thắng -> Th(ắ)ng, Hảo -> H(ảo)
def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1

    # duyệt từng ký tự trong word
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1)) # x xác định nguyên âm nào còn y sẽ xác định dấu ứng với nguyên âm đó
        if x != -1: # nếu char là nguyên âm
            if nguyen_am_index == -1: # nếu chưa tìm thấy nguyên âm
                nguyen_am_index = index
            else:                     # đã tìm thấy nguyên âm
                if index - nguyen_am_index != 1: # nếu nguyên âm tìm thấy ko nằm cạnh nguyên âm đã tìm thấy
                    return False
                nguyen_am_index = index
    return True

print("Nguyene: ", is_valid_vietnam_word("Nguyene"))
print('Nguyễn: ', is_valid_vietnam_word('Nguyễn'))

Nguyene:  False
Nguyễn:  True


In [ ]:
# Thực hiện chuẩn hóa dấu cho từng từ trong tiếng Việt
def chuan_hoa_dau_tu_tieng_viet(word):
    # Nếu là invalid word thì sẽ không xử lý bao gồm từ tiếng anh, số, từ gõ sai, ...
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False

    # Duyệt qua từng ký tự
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1: # Nếu char là phụ âm
            continue

        elif x == 9:  # check 'qu'
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True

        elif x == 5:  # check 'gi'
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True

        if y != 0:  # nếu có dấu
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0] # bỏ dấu

        # Nếu không phải trường hợp 'qu' hoặc 'gi' hoặc là index khác 1 (có nguyên âm từ ký tự thứ 3 trở đi)
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)

    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)

# Thực hiện chuẩn hóa cho cả câu tiếng Việt và đưa câu đó về dạng viết thường
def chuan_hoa_dau_cau_tieng_viet(sentence):
    sentence = sentence.lower() # Chuyển về dạng lowercase
    words = sentence.split()

    # thực hiện xử lý trên từng từ
    for index, word in enumerate(words):
        cw = re.sub(r'^([b-dđghklmnpqrstv-xzB-DĐGHKLMNPQRSTV-XZ]*)([aăâeêioôơuưy]*[aăâeêioôơuưy]{1}[aăâeêioôơuưy]*)([b-dđghklmnpqrstv-xzB-DĐGHKLMNPQRSTV-XZ]*)$', r'\1/\2/\3', word).split('/')
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

print(chuan_hoa_dau_cau_tieng_viet('đường mật có hại không? không Ai BiẾt. Chúng TôI cHỉ BiếT là KiẾn thích đườNg'))

đường mật có hại không? không ai biết. chúng tôi chỉ biết là kiến thích đường


In [ ]:
'''
Tách từ tiếng việt:
  - Đơn vị từ có từ đơn & từ ghép => tăng chính xác đáng kể khi không cần học sự liên kết giữa 2 từ trong 1 từ ghép
  - Nối 2 từ lại
  => sử dụng undersea
'''
sentence = 'Có một con bò ở trên cành cây.'
print(word_tokenize(sentence))
print(word_tokenize(sentence, format='text'))

['Có', 'một', 'con', 'bò', 'ở', 'trên', 'cành', 'cây', '.']
Có một con bò ở trên cành cây .


In [ ]:
'''
Xóa các ký tự không cần thiết:
  - Giảm chiều đặc trưng, tăng tốc độ học, xử lý
  - Tránh gây nhiễu cho mô hình
Loại bỏ những ký tự không cần thiết như dấu ngắt câu '\n' và các ký tự đặc biệt.
'''
def remove_unnecessary_charactor(document):
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

In [ ]:
def text_preprocessing(document):
  # xóa html
  document = remove_html(document)
  # chuẩn hóa unicode
  document = convert_unicode(document)
  # chuẩn hóa các gõ dấu và đưa về dạng viết thường
  document = chuan_hoa_dau_cau_tieng_viet(document)
  # tách từ
  document = word_tokenize(document, format='text')
  # xóa bỏ ký tự không cần thiết
  document = remove_unnecessary_charactor(document)
  return document

In [ ]:
text_preprocessing(df.text[0])

'sau khi hđqt chính_thức ban_hành sơ_đồ cơ_cấu tổ_chức mới của vpbank khối khcn sme đã hoạt_động từ tháng 72011'

In [ ]:
test_df['text'] = test_df['text'].apply(text_preprocessing)

In [ ]:
'''
Import Vietnamese stopwords: '/content/drive/MyDrive/Research - CSR_NLP/Code/vietnamese-stopwords-dash.txt'
'''
def import_file(filename):
    with open(filename, 'r') as file:
        content = file.readlines()
    # Remove trailing newline characters
    content = [line.rstrip('\n') for line in content]
    return content

# Provide the filename of the text file you want to import
file_name = '/content/drive/MyDrive/Research - CSR_NLP/Code/vietnamese-stopwords-dash.txt'
file_contents = import_file(file_name)

# Now you can use the file_contents list containing the lines of the text file
print(file_contents)


['và', 'các', 'của', 'trong', 'cho', 'được', 'với', 'đã', 'cid', 'những', 'đó', 'sự', 'cũng', 'này', 'sẽ', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'mà', 'hoặc']


In [ ]:
stopwords = set()
for word in file_contents:
  stopwords.add(word)

stopwords.add('đến')

In [ ]:
import re

def remove_stopwords(document):
    words = document.split(' ')
    res = []
    for word in words:
        # Remove numbers from the word using regex
        word_without_numbers = re.sub(r'\d', '', word)
        if word_without_numbers not in stopwords:
            res.append(word_without_numbers)
    return ' '.join(res)

In [ ]:
print('Processing testing set ... ')
for i, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    test_df.at[i, 'text'] = remove_stopwords(row['text'])

Processing testing set ... 


  0%|          | 0/3330 [00:00<?, ?it/s]

In [ ]:
def prob_1(df):
    df_cop = df.copy()
    df_cop.loc[df_cop['label'] != 'No CSR', 'label_1'] = 'CSR'
    df_cop.loc[df_cop['label'] == 'No CSR', 'label_1'] = 'No CSR'

    return df_cop


def prob_1_label(df):
    df_cop = df.copy()
    df_cop['label_1'] = df_cop['label_1'].replace({
        'No CSR': 0,
        'CSR': 1
    })
    df_cop = df_cop.reset_index(drop = True)
    return df_cop

In [ ]:
def prob_2(df):
    '''
    Second problem: Classify to 6 type of CSR
    '''
    df_cop = df.copy()
    pattern_mapping = {
        r"CSR0_\d+": "CSR0",
        r"CSR1_\d+": "CSR1",
        r"CSR2_\d+": "CSR2",
        r"CSR3_\d+": "CSR3",
        r"CSR4_\d+": "CSR4",
        r"CSR5_\d+": "CSR5"
    }
    # Apply the pattern mapping to transform the label values
    df_cop["label_2"] = df_cop["label"].replace(pattern_mapping, regex=True)
    df_cop['text'] = df_cop['text'].str.replace(r'\s+', ' ')
    return df_cop

def prob_2_label(df):
    df_cop = df.copy()
    df_cop['label_2'] = df_cop['label_2'].replace({
        "CSR0": 0,
        "CSR1": 1,
        "CSR2": 2,
        "CSR3": 3,
        "CSR4": 4,
        "CSR5": 5
    })
    df_cop.loc[df_cop['label'] != 'No CSR', 'label_2'] = df_cop['label_2']
    df_cop = df_cop.reset_index(drop=True)
    return df_cop


In [ ]:
test_df = prob_1_label(prob_1(test_df))
test_df

,name,year,text,label,label_1
0,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2016,sacombank ban_hành bộ tiêu_chuẩn năng_lực quy_...,CSR0_8,1
1,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2020,sacombank hướng sử_dụng nguyên_vật_liệu theo t...,CSR2_6,1
2,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2020,hệ_thống xử_lý nước_thải seabank đáp_ứng tiêu_...,CSR2_1,1
3,30. NGÂN HÀNG TMCP ĐẠI CHÚNG VIỆT NAM,2013,tình_thương có sức lan tỏa tự_nhiên vì_vậy khô...,CSR5_0,1
4,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2017,hiện_nay bộ chỉ_tiêu kpis phát_huy tốt vai_trò...,CSR0_10,1
...,...,...,...,...,...
3325,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,trên thực_tế thành_viên hội_đồng_quản_trị thàn...,CSR0_1,1
3326,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,nhân_dịp chủ_tịch nước trương tấn sang tặng nh...,CSR5_3,1
3327,2. NGÂN HÀNG TMCP KỸ THƯƠNG VIỆT NAM,2021,đồng_thời ông có kinh_nghiệm tham_gia quản_lý ...,No CSR,0
3328,27. NGÂN HÀNG TMCP CÔNG THƯƠNG VIỆT NAM,2016,ngoài ra vietinbank đầu_tư hỗ_trợ xây_dựng khu...,CSR5_4,1


In [ ]:
test_df = prob_2_label(prob_2(test_df))
test_df

<ipython-input-58-4b93e42ac06e>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df_cop['text'] = df_cop['text'].str.replace(r'\s+', ' ')


,name,year,text,label,label_1,label_2
0,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2016,sacombank ban_hành bộ tiêu_chuẩn năng_lực quy_...,CSR0_8,1,0
1,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2020,sacombank hướng sử_dụng nguyên_vật_liệu theo t...,CSR2_6,1,2
2,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2020,hệ_thống xử_lý nước_thải seabank đáp_ứng tiêu_...,CSR2_1,1,2
3,30. NGÂN HÀNG TMCP ĐẠI CHÚNG VIỆT NAM,2013,tình_thương có sức lan tỏa tự_nhiên vì_vậy khô...,CSR5_0,1,5
4,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2017,hiện_nay bộ chỉ_tiêu kpis phát_huy tốt vai_trò...,CSR0_10,1,0
...,...,...,...,...,...,...
3325,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,trên thực_tế thành_viên hội_đồng_quản_trị thàn...,CSR0_1,1,0
3326,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,nhân_dịp chủ_tịch nước trương tấn sang tặng nh...,CSR5_3,1,5
3327,2. NGÂN HÀNG TMCP KỸ THƯƠNG VIỆT NAM,2021,đồng_thời ông có kinh_nghiệm tham_gia quản_lý ...,No CSR,0,No CSR
3328,27. NGÂN HÀNG TMCP CÔNG THƯƠNG VIỆT NAM,2016,ngoài ra vietinbank đầu_tư hỗ_trợ xây_dựng khu...,CSR5_4,1,5


In [ ]:
test_df['label_3'] = test_df['label'].str.split('_').str[1]
test_df['label_3'] = test_df['label_3'].fillna('No CSR')
test_df

,name,year,text,label,label_1,label_2,label_3
0,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2016,sacombank ban_hành bộ tiêu_chuẩn năng_lực quy_...,CSR0_8,1,0,8
1,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2020,sacombank hướng sử_dụng nguyên_vật_liệu theo t...,CSR2_6,1,2,6
2,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2020,hệ_thống xử_lý nước_thải seabank đáp_ứng tiêu_...,CSR2_1,1,2,1
3,30. NGÂN HÀNG TMCP ĐẠI CHÚNG VIỆT NAM,2013,tình_thương có sức lan tỏa tự_nhiên vì_vậy khô...,CSR5_0,1,5,0
4,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2017,hiện_nay bộ chỉ_tiêu kpis phát_huy tốt vai_trò...,CSR0_10,1,0,10
...,...,...,...,...,...,...,...
3325,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,trên thực_tế thành_viên hội_đồng_quản_trị thàn...,CSR0_1,1,0,1
3326,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,nhân_dịp chủ_tịch nước trương tấn sang tặng nh...,CSR5_3,1,5,3
3327,2. NGÂN HÀNG TMCP KỸ THƯƠNG VIỆT NAM,2021,đồng_thời ông có kinh_nghiệm tham_gia quản_lý ...,No CSR,0,No CSR,No CSR
3328,27. NGÂN HÀNG TMCP CÔNG THƯƠNG VIỆT NAM,2016,ngoài ra vietinbank đầu_tư hỗ_trợ xây_dựng khu...,CSR5_4,1,5,4


In [ ]:
prediction_df = test_df.copy()

In [ ]:
# Load BPE encoder
parser = argparse.ArgumentParser()
parser.add_argument(
    '--bpe-codes',
    default='/content/drive/MyDrive/Research - CSR_NLP/Code/PhoBERT_base_transformers/PhoBERT_base_transformers/bpe.codes',
    required=False,
    type=str,
    help='path to fastBPE BPE'
)

args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file('/content/drive/MyDrive/Research - CSR_NLP/Code/PhoBERT_base_transformers/PhoBERT_base_transformers/dict.txt')

# Case 1: Problem 1

In [ ]:
# load model phoBert và tokenizer của model đó
phoBert = AutoModel.from_pretrained('vinai/phobert-base')
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
test_x = test_df['text']
test_y = test_df['label_1']

In [ ]:
# Đưa từng sentence qua tokenizer của PhoBERT để convert sang dạng token index với cùng chiều dài
# params
MAX_SEQ_LEN = 256 # chiều dài tối đa của một câu
# id của 1 số token đặc biệt
cls_id = 0  # đầu câu
eos_id = 2  # cuối câu
pad_id = 1  # padding

# Hàm xử lý dữ liệu trên từng sentence
def tokenize_line(line):
    tokenized = tokenizer.encode(line)

    l = len(tokenized)
    if l > MAX_SEQ_LEN: # nếu dài hơn thì cắt bỏ
        tokenized = tokenized[:MAX_SEQ_LEN]
        tokenized[-1] = eos_id # thêm EOS vào cuối câu
    else: # nếu ngắn hơn thì thêm padding vào
        tokenized = tokenized + [pad_id, ] * (MAX_SEQ_LEN - l)

    return tokenized

In [ ]:
# Thực hiện chuyển dataset ban đầu thành subword index
MAX_LEN = 256

test_ids = []
print('Processing text set ...')
for sent in tqdm(test_x):
  subwords = ' ' + bpe.encode(sent) + ' '
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
  test_ids.append(encoded_sent)

Processing text set ...


  0%|          | 0/3330 [00:00<?, ?it/s]

In [ ]:
# thêm padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype='long', value=1, truncating='post', padding='post')

In [ ]:
# Tạo mask để BERT không chú ý tới padding (pad id = 1)
test_masks = []
for sent in test_ids:
  mask = [int(token_id != 1) for token_id in sent]
  test_masks.append(mask)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 4

# convert sang tensor
test_inputs = torch.tensor(test_ids)
test_labels = torch.tensor(test_y)
test_masks = torch.tensor(test_masks)

# tạo data loader để train
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=BATCH_SIZE)

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW


NUM_CLASSES = 2

# load file config
config_pr1 = RobertaConfig.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 1: Binary/Bert Fine-tune/config.json',
    from_tf=False,
    num_labels=NUM_CLASSES,
    output_hidden_states=False
)

# load model
BERT_SA_pr1 = RobertaForSequenceClassification.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 1: Binary/Bert Fine-tune/pytorch_model.bin',
    config = config_pr1
)

In [ ]:
BERT_SA_pr1.cuda()
print('Done')

Done


In [ ]:
device = 'cuda'

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Set the model to evaluation mode
BERT_SA_pr1.eval()

# Initialize empty lists to store predicted labels and true labels
predicted_labels = []
true_labels = []

# Predict the labels for the test data
for batch in tqdm(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = BERT_SA_pr1(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask
        )
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Convert logits to class labels
        batch_predicted_labels = np.argmax(logits, axis=1)
        predicted_labels.extend(batch_predicted_labels)
        true_labels.extend(label_ids)

# Attach the predicted labels to the test_df dataframe
test_df['predicted_label_1'] = predicted_labels

# Print the classification report
print(classification_report(true_labels, predicted_labels))

# Print the confusion matrix
confusion_mat = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(confusion_mat)


  0%|          | 0/833 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.72      0.95      0.82      1128
           1       0.97      0.81      0.88      2202

    accuracy                           0.86      3330
   macro avg       0.84      0.88      0.85      3330
weighted avg       0.88      0.86      0.86      3330

Confusion Matrix:
[[1066   62]
 [ 420 1782]]


In [ ]:
def add_id_column(df):
    df_cop = df.copy()
    num_rows = len(df_cop)
    ids = np.arange(1, num_rows + 1)  # Generate unique IDs
    df_cop['ID'] = ids  # Add the IDs as a new column
    return df_cop

test_df_with_id = add_id_column(test_df)
test_df_with_id = test_df_with_id[['ID', 'name', 'year', 'text', 'label', 'label_1', 'label_2', 'label_3', 'predicted_label_1']]
test_df_with_id.head()

,ID,name,year,text,label,label_1,label_2,label_3,predicted_label_1
0,1,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2016,sacombank ban_hành bộ tiêu_chuẩn năng_lực quy_...,CSR0_8,1,0,8,1
1,2,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2020,sacombank hướng sử_dụng nguyên_vật_liệu theo t...,CSR2_6,1,2,6,1
2,3,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2020,hệ_thống xử_lý nước_thải seabank đáp_ứng tiêu_...,CSR2_1,1,2,1,1
3,4,30. NGÂN HÀNG TMCP ĐẠI CHÚNG VIỆT NAM,2013,tình_thương có sức lan tỏa tự_nhiên vì_vậy khô...,CSR5_0,1,5,0,1
4,5,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2017,hiện_nay bộ chỉ_tiêu kpis phát_huy tốt vai_trò...,CSR0_10,1,0,10,0


In [ ]:
test_df.predicted_label_1.value_counts()

1    1844
0    1486
Name: predicted_label_1, dtype: int64

Nếu predict_label_1 = 1 => tiếp tục

In [ ]:
def drop_rows(df):
    df_cop = df.copy()
    df_cop = df_cop[df_cop['predicted_label_1'] != 0]  # Drop rows with predicted_label_1 equal to 0
    df_cop = df_cop.reset_index(drop=True)  # Reset the index after dropping rows
    return df_cop

test_df_pr2 = drop_rows(test_df_with_id)
test_df_pr2

,ID,name,year,text,label,label_1,label_2,label_3,predicted_label_1
0,1,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2016,sacombank ban_hành bộ tiêu_chuẩn năng_lực quy_...,CSR0_8,1,0,8,1
1,2,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2020,sacombank hướng sử_dụng nguyên_vật_liệu theo t...,CSR2_6,1,2,6,1
2,3,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2020,hệ_thống xử_lý nước_thải seabank đáp_ứng tiêu_...,CSR2_1,1,2,1,1
3,4,30. NGÂN HÀNG TMCP ĐẠI CHÚNG VIỆT NAM,2013,tình_thương có sức lan tỏa tự_nhiên vì_vậy khô...,CSR5_0,1,5,0,1
4,6,22. NGÂN HÀNG TMCP XĂNG DẦU PETROLIMEX,2011,đặc_biệt pg bank trực_tiếp tổ_chức nhiều chươn...,CSR5_0,1,5,0,1
...,...,...,...,...,...,...,...,...,...
1839,3319,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2021,khi nhân_sự tăng lên kéo_theo gia_tăng trang_t...,CSR2_6,1,2,6,1
1840,3323,6. NGÂN HÀNG TMCP PHÁT TRIỂN TP. HỒ CHI...,2021,chỉ riêng năm hdbank tài_trợ suất học_bổng trẻ...,CSR5_0,1,5,0,1
1841,3326,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,trên thực_tế thành_viên hội_đồng_quản_trị thàn...,CSR0_1,1,0,1,1
1842,3327,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,nhân_dịp chủ_tịch nước trương tấn sang tặng nh...,CSR5_3,1,5,3,1


# Case 2: Problem 2

In [ ]:
test_x = test_df_pr2['text']
test_y = test_df_pr2['label_2']

In [ ]:
# Đưa từng sentence qua tokenizer của PhoBERT để convert sang dạng token index với cùng chiều dài
# params
MAX_SEQ_LEN = 256 # chiều dài tối đa của một câu
# id của 1 số token đặc biệt
cls_id = 0  # đầu câu
eos_id = 2  # cuối câu
pad_id = 1  # padding

# Hàm xử lý dữ liệu trên từng sentence
def tokenize_line(line):
    tokenized = tokenizer.encode(line)

    l = len(tokenized)
    if l > MAX_SEQ_LEN: # nếu dài hơn thì cắt bỏ
        tokenized = tokenized[:MAX_SEQ_LEN]
        tokenized[-1] = eos_id # thêm EOS vào cuối câu
    else: # nếu ngắn hơn thì thêm padding vào
        tokenized = tokenized + [pad_id, ] * (MAX_SEQ_LEN - l)

    return tokenized

In [ ]:
# Thực hiện chuyển dataset ban đầu thành subword index
MAX_LEN = 256

test_ids = []
print('Processing text set ...')
for sent in tqdm(test_x):
  subwords = ' ' + bpe.encode(sent) + ' '
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
  test_ids.append(encoded_sent)

Processing text set ...


  0%|          | 0/1844 [00:00<?, ?it/s]

In [ ]:
# thêm padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype='long', value=1, truncating='post', padding='post')

In [ ]:
# Tạo mask để BERT không chú ý tới padding (pad id = 1)
test_masks = []
for sent in test_ids:
  mask = [int(token_id != 1) for token_id in sent]
  test_masks.append(mask)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 4

# convert sang tensor
test_inputs = torch.tensor(test_ids)
# test_labels = torch.tensor(test_y)
test_masks = torch.tensor(test_masks)

# tạo data loader để train
test_data = TensorDataset(test_inputs, test_masks)#, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=BATCH_SIZE)

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW


NUM_CLASSES = 6

# load file config
config_pr2 = RobertaConfig.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 2: CSR Topic/Bert Fine-tune/config.json',
    from_tf=False,
    num_labels=NUM_CLASSES,
    output_hidden_states=False
)

# load model
BERT_SA_pr2 = RobertaForSequenceClassification.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 2: CSR Topic/Bert Fine-tune/pytorch_model.bin',
    config = config_pr2
)

In [ ]:
BERT_SA_pr2.cuda()
print('Done')

Done


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Set the model to evaluation mode
BERT_SA_pr2.eval()

# Initialize empty lists to store predicted labels and true labels
predicted_labels = []
true_labels = []

# Predict the labels for the test data
for batch in tqdm(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        outputs = BERT_SA_pr2(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask
        )
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        # label_ids = b_labels.to('cpu').numpy()

        # Convert logits to class labels
        batch_predicted_labels = np.argmax(logits, axis=1)
        predicted_labels.extend(batch_predicted_labels)
        # true_labels.extend(label_ids)

# Attach the predicted labels to the test_df dataframe
# test_df_with_id['predicted_label_2'] = predicted_labels
test_df_pr2['predicted_label_2'] = predicted_labels

# Print the classification report
# print(classification_report(true_labels, predicted_labels))

# # Print the confusion matrix
# confusion_mat = confusion_matrix(true_labels, predicted_labels)
# print("Confusion Matrix:")
# print(confusion_mat)

  0%|          | 0/461 [00:00<?, ?it/s]

In [ ]:
test_df_pr2.head()

,ID,name,year,text,label,label_1,label_2,label_3,predicted_label_1,predicted_label_2
0,1,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2016,sacombank ban_hành bộ tiêu_chuẩn năng_lực quy_...,CSR0_8,1,0,8,1,0
1,2,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2020,sacombank hướng sử_dụng nguyên_vật_liệu theo t...,CSR2_6,1,2,6,1,2
2,3,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2020,hệ_thống xử_lý nước_thải seabank đáp_ứng tiêu_...,CSR2_1,1,2,1,1,2
3,4,30. NGÂN HÀNG TMCP ĐẠI CHÚNG VIỆT NAM,2013,tình_thương có sức lan tỏa tự_nhiên vì_vậy khô...,CSR5_0,1,5,0,1,5
4,6,22. NGÂN HÀNG TMCP XĂNG DẦU PETROLIMEX,2011,đặc_biệt pg bank trực_tiếp tổ_chức nhiều chươn...,CSR5_0,1,5,0,1,5


In [ ]:
test_df_pr2_0 = test_df_pr2[test_df_pr2['predicted_label_2'] == 0]
test_df_pr2_1 = test_df_pr2[test_df_pr2['predicted_label_2'] == 1]
test_df_pr2_2 = test_df_pr2[test_df_pr2['predicted_label_2'] == 2]
test_df_pr2_3 = test_df_pr2[test_df_pr2['predicted_label_2'] == 3]
test_df_pr2_4 = test_df_pr2[test_df_pr2['predicted_label_2'] == 4]
test_df_pr2_5 = test_df_pr2[test_df_pr2['predicted_label_2'] == 5]

# Case 3: Problem 3

### Class 0

In [ ]:
test_x = test_df_pr2_0['text']
# test_y = test_df_pr2_0['label_2']

In [ ]:
# Đưa từng sentence qua tokenizer của PhoBERT để convert sang dạng token index với cùng chiều dài
# params
MAX_SEQ_LEN = 256 # chiều dài tối đa của một câu
# id của 1 số token đặc biệt
cls_id = 0  # đầu câu
eos_id = 2  # cuối câu
pad_id = 1  # padding

# Hàm xử lý dữ liệu trên từng sentence
def tokenize_line(line):
    tokenized = tokenizer.encode(line)

    l = len(tokenized)
    if l > MAX_SEQ_LEN: # nếu dài hơn thì cắt bỏ
        tokenized = tokenized[:MAX_SEQ_LEN]
        tokenized[-1] = eos_id # thêm EOS vào cuối câu
    else: # nếu ngắn hơn thì thêm padding vào
        tokenized = tokenized + [pad_id, ] * (MAX_SEQ_LEN - l)

    return tokenized

In [ ]:
# Thực hiện chuyển dataset ban đầu thành subword index
MAX_LEN = 256

test_ids = []
print('Processing text set ...')
for sent in tqdm(test_x):
  subwords = ' ' + bpe.encode(sent) + ' '
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
  test_ids.append(encoded_sent)

Processing text set ...


  0%|          | 0/621 [00:00<?, ?it/s]

In [ ]:
# thêm padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype='long', value=1, truncating='post', padding='post')

In [ ]:
# Tạo mask để BERT không chú ý tới padding (pad id = 1)
test_masks = []
for sent in test_ids:
  mask = [int(token_id != 1) for token_id in sent]
  test_masks.append(mask)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 4

# convert sang tensor
test_inputs = torch.tensor(test_ids)
# test_labels = torch.tensor(test_y)
test_masks = torch.tensor(test_masks)

# tạo data loader để train
test_data = TensorDataset(test_inputs, test_masks)#, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=BATCH_SIZE)

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

NUM_CLASSES = 12

# load file config
config_pr2_0 = RobertaConfig.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 3: CSR Sub Topic/CSR0/Bert Fine-tune/config.json',
    from_tf=False,
    num_labels=NUM_CLASSES,
    output_hidden_states=False
)

# load model
BERT_SA_pr2_0 = RobertaForSequenceClassification.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 3: CSR Sub Topic/CSR0/Bert Fine-tune/pytorch_model.bin',
    config = config_pr2_0
)

In [ ]:
BERT_SA_pr2_0.cuda()
print('Done')

Done


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Set the model to evaluation mode
BERT_SA_pr2_0.eval()

# Initialize empty lists to store predicted labels and true labels
predicted_labels = []
true_labels = []

# Predict the labels for the test data
for batch in tqdm(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        outputs = BERT_SA_pr2_0(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask
        )
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        # label_ids = b_labels.to('cpu').numpy()

        # Convert logits to class labels
        batch_predicted_labels = np.argmax(logits, axis=1)
        predicted_labels.extend(batch_predicted_labels)
        # true_labels.extend(label_ids)

# Attach the predicted labels to the test_df dataframe
# test_df_with_id['predicted_label_2'] = predicted_labels
test_df_pr2_0['predicted_label_3'] = predicted_labels

# Print the classification report
# print(classification_report(true_labels, predicted_labels))

# # Print the confusion matrix
# confusion_mat = confusion_matrix(true_labels, predicted_labels)
# print("Confusion Matrix:")
# print(confusion_mat)

  0%|          | 0/156 [00:00<?, ?it/s]

<ipython-input-115-e91cd1e58ad8>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_pr2_0['predicted_label_3'] = predicted_labels


In [ ]:
test_df_pr2_0.head()

,ID,name,year,text,label,label_1,label_2,label_3,predicted_label_1,predicted_label_2,predicted_label_3
0,1,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2016,sacombank ban_hành bộ tiêu_chuẩn năng_lực quy_...,CSR0_8,1,0,8,1,0,8
5,8,27. NGÂN HÀNG TMCP CÔNG THƯƠNG VIỆT NAM,2018,hoạt_động đảm_bảo tất_cả người lao_động hệ_thố...,CSR0_2,1,0,2,1,0,2
8,14,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2022,theo lãnh_đạo sacombank có_thể nói cùng chính_...,CSR0_5,1,0,5,1,0,1
11,19,6. NGÂN HÀNG TMCP PHÁT TRIỂN TP. HỒ CHI...,2022,hdbank trả lương người lao_động theo quy_chế t...,CSR0_1,1,0,1,1,0,1
13,23,2. NGÂN HÀNG TMCP KỸ THƯƠNG VIỆT NAM,2019,triển_khai thành_công khóa học thi lượt học_vi...,CSR0_8,1,0,8,1,0,8


### Class 1

In [ ]:
test_x = test_df_pr2_1['text']
# test_y = test_df_pr2_0['label_2']

In [ ]:
# Đưa từng sentence qua tokenizer của PhoBERT để convert sang dạng token index với cùng chiều dài
# params
MAX_SEQ_LEN = 256 # chiều dài tối đa của một câu
# id của 1 số token đặc biệt
cls_id = 0  # đầu câu
eos_id = 2  # cuối câu
pad_id = 1  # padding

# Hàm xử lý dữ liệu trên từng sentence
def tokenize_line(line):
    tokenized = tokenizer.encode(line)

    l = len(tokenized)
    if l > MAX_SEQ_LEN: # nếu dài hơn thì cắt bỏ
        tokenized = tokenized[:MAX_SEQ_LEN]
        tokenized[-1] = eos_id # thêm EOS vào cuối câu
    else: # nếu ngắn hơn thì thêm padding vào
        tokenized = tokenized + [pad_id, ] * (MAX_SEQ_LEN - l)

    return tokenized

In [ ]:
# Thực hiện chuyển dataset ban đầu thành subword index
MAX_LEN = 256

test_ids = []
print('Processing text set ...')
for sent in tqdm(test_x):
  subwords = ' ' + bpe.encode(sent) + ' '
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
  test_ids.append(encoded_sent)

Processing text set ...


  0%|          | 0/88 [00:00<?, ?it/s]

In [ ]:
# thêm padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype='long', value=1, truncating='post', padding='post')

In [ ]:
# Tạo mask để BERT không chú ý tới padding (pad id = 1)
test_masks = []
for sent in test_ids:
  mask = [int(token_id != 1) for token_id in sent]
  test_masks.append(mask)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 4

# convert sang tensor
test_inputs = torch.tensor(test_ids)
# test_labels = torch.tensor(test_y)
test_masks = torch.tensor(test_masks)

# tạo data loader để train
test_data = TensorDataset(test_inputs, test_masks)#, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=BATCH_SIZE)

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

NUM_CLASSES = 5

# load file config
config_pr2_1 = RobertaConfig.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 3: CSR Sub Topic/CSR1/Bert Fine-tune/config.json',
    from_tf=False,
    num_labels=NUM_CLASSES,
    output_hidden_states=False
)

# load model
BERT_SA_pr2_1 = RobertaForSequenceClassification.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 3: CSR Sub Topic/CSR1/Bert Fine-tune/pytorch_model.bin',
    config = config_pr2_1
)

In [ ]:
BERT_SA_pr2_1.cuda()
print('Done')

Done


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Set the model to evaluation mode
BERT_SA_pr2_1.eval()

# Initialize empty lists to store predicted labels and true labels
predicted_labels = []
true_labels = []

# Predict the labels for the test data
for batch in tqdm(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        outputs = BERT_SA_pr2_1(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask
        )
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        # label_ids = b_labels.to('cpu').numpy()

        # Convert logits to class labels
        batch_predicted_labels = np.argmax(logits, axis=1)
        predicted_labels.extend(batch_predicted_labels)
        # true_labels.extend(label_ids)

# Attach the predicted labels to the test_df dataframe
# test_df_with_id['predicted_label_2'] = predicted_labels
test_df_pr2_1['predicted_label_3'] = predicted_labels

# Print the classification report
# print(classification_report(true_labels, predicted_labels))

# # Print the confusion matrix
# confusion_mat = confusion_matrix(true_labels, predicted_labels)
# print("Confusion Matrix:")
# print(confusion_mat)

  0%|          | 0/22 [00:00<?, ?it/s]

<ipython-input-127-e1026fc2c99c>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_pr2_1['predicted_label_3'] = predicted_labels


In [ ]:
test_df_pr2_1.head()

,ID,name,year,text,label,label_1,label_2,label_3,predicted_label_1,predicted_label_2,predicted_label_3
18,30,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2022,ngoài_ra nội_dung đào_tạo về phòng chống tham_...,CSR1_2,1,1,2,1,1,2
29,61,10. NGÂN HÀNG TMCP HÀNG HẢI VIỆT NAM,2017,quản_trị rủi_ro theo thông_lệ quốc_tế như base...,CSR1_1,1,1,1,1,1,1
61,109,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2020,phòng_chống tham_nhũng tội_phạm rửa tiền là mố...,CSR1_2,1,1,2,1,1,2
83,156,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2018,vcb thực_hiện trích nộp bảo_hiểm xã_hội bảo_hi...,CSR0_5,1,0,5,1,1,1
109,200,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2017,nhận_thức rõ điều năm vcb nghiêm_túc thực_hiện...,CSR1_0,1,1,0,1,1,0


### Class 2

In [ ]:
test_x = test_df_pr2_2['text']
# test_y = test_df_pr2_0['label_2']

In [ ]:
# Đưa từng sentence qua tokenizer của PhoBERT để convert sang dạng token index với cùng chiều dài
# params
MAX_SEQ_LEN = 256 # chiều dài tối đa của một câu
# id của 1 số token đặc biệt
cls_id = 0  # đầu câu
eos_id = 2  # cuối câu
pad_id = 1  # padding

# Hàm xử lý dữ liệu trên từng sentence
def tokenize_line(line):
    tokenized = tokenizer.encode(line)

    l = len(tokenized)
    if l > MAX_SEQ_LEN: # nếu dài hơn thì cắt bỏ
        tokenized = tokenized[:MAX_SEQ_LEN]
        tokenized[-1] = eos_id # thêm EOS vào cuối câu
    else: # nếu ngắn hơn thì thêm padding vào
        tokenized = tokenized + [pad_id, ] * (MAX_SEQ_LEN - l)

    return tokenized

In [ ]:
# Thực hiện chuyển dataset ban đầu thành subword index
MAX_LEN = 256

test_ids = []
print('Processing text set ...')
for sent in tqdm(test_x):
  subwords = ' ' + bpe.encode(sent) + ' '
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
  test_ids.append(encoded_sent)

Processing text set ...


  0%|          | 0/191 [00:00<?, ?it/s]

In [ ]:
# thêm padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype='long', value=1, truncating='post', padding='post')

In [ ]:
# Tạo mask để BERT không chú ý tới padding (pad id = 1)
test_masks = []
for sent in test_ids:
  mask = [int(token_id != 1) for token_id in sent]
  test_masks.append(mask)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 4

# convert sang tensor
test_inputs = torch.tensor(test_ids)
# test_labels = torch.tensor(test_y)
test_masks = torch.tensor(test_masks)

# tạo data loader để train
test_data = TensorDataset(test_inputs, test_masks)#, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=BATCH_SIZE)

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

NUM_CLASSES = 11

# load file config
config_pr2_2 = RobertaConfig.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 3: CSR Sub Topic/CSR2/Bert Fine-tune/config.json',
    from_tf=False,
    num_labels=NUM_CLASSES,
    output_hidden_states=False
)

# load model
BERT_SA_pr2_2 = RobertaForSequenceClassification.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 3: CSR Sub Topic/CSR2/Bert Fine-tune/pytorch_model.bin',
    config = config_pr2_2
)

In [ ]:
BERT_SA_pr2_2.cuda()
print('Done')

Done


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Set the model to evaluation mode
BERT_SA_pr2_2.eval()

# Initialize empty lists to store predicted labels and true labels
predicted_labels = []
true_labels = []

# Predict the labels for the test data
for batch in tqdm(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        outputs = BERT_SA_pr2_2(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask
        )
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        # label_ids = b_labels.to('cpu').numpy()

        # Convert logits to class labels
        batch_predicted_labels = np.argmax(logits, axis=1)
        predicted_labels.extend(batch_predicted_labels)
        # true_labels.extend(label_ids)

# Attach the predicted labels to the test_df dataframe
# test_df_with_id['predicted_label_2'] = predicted_labels
test_df_pr2_2['predicted_label_3'] = predicted_labels

# Print the classification report
# print(classification_report(true_labels, predicted_labels))

# # Print the confusion matrix
# confusion_mat = confusion_matrix(true_labels, predicted_labels)
# print("Confusion Matrix:")
# print(confusion_mat)

  0%|          | 0/48 [00:00<?, ?it/s]

<ipython-input-137-3c5e4077ec66>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_pr2_2['predicted_label_3'] = predicted_labels


In [ ]:
test_df_pr2_2.head()

,ID,name,year,text,label,label_1,label_2,label_3,predicted_label_1,predicted_label_2,predicted_label_3
1,2,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2020,sacombank hướng sử_dụng nguyên_vật_liệu theo t...,CSR2_6,1,2,6,1,2,6
2,3,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2020,hệ_thống xử_lý nước_thải seabank đáp_ứng tiêu_...,CSR2_1,1,2,1,1,2,1
7,12,27. NGÂN HÀNG TMCP CÔNG THƯƠNG VIỆT NAM,2022,kết_quả triển_khai thực_hiện biện_pháp tiết_ki...,CSR2_6,1,2,6,1,2,6
16,27,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2021,phù_hợp xu_thế chung nền kinh_tế nhằm góp_phần...,CSR5_0,1,5,0,1,2,6
20,33,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2020,nước sinh_hoạt văn_phòng gom xử_lý công_suất m...,CSR2_1,1,2,1,1,2,1


### Class 5

In [ ]:
test_x = test_df_pr2_5['text']
# test_y = test_df_pr2_0['label_2']

In [ ]:
# Đưa từng sentence qua tokenizer của PhoBERT để convert sang dạng token index với cùng chiều dài
# params
MAX_SEQ_LEN = 256 # chiều dài tối đa của một câu
# id của 1 số token đặc biệt
cls_id = 0  # đầu câu
eos_id = 2  # cuối câu
pad_id = 1  # padding

# Hàm xử lý dữ liệu trên từng sentence
def tokenize_line(line):
    tokenized = tokenizer.encode(line)

    l = len(tokenized)
    if l > MAX_SEQ_LEN: # nếu dài hơn thì cắt bỏ
        tokenized = tokenized[:MAX_SEQ_LEN]
        tokenized[-1] = eos_id # thêm EOS vào cuối câu
    else: # nếu ngắn hơn thì thêm padding vào
        tokenized = tokenized + [pad_id, ] * (MAX_SEQ_LEN - l)

    return tokenized

In [ ]:
# Thực hiện chuyển dataset ban đầu thành subword index
MAX_LEN = 256

test_ids = []
print('Processing text set ...')
for sent in tqdm(test_x):
  subwords = ' ' + bpe.encode(sent) + ' '
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
  test_ids.append(encoded_sent)

Processing text set ...


  0%|          | 0/895 [00:00<?, ?it/s]

In [ ]:
# thêm padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype='long', value=1, truncating='post', padding='post')

In [ ]:
# Tạo mask để BERT không chú ý tới padding (pad id = 1)
test_masks = []
for sent in test_ids:
  mask = [int(token_id != 1) for token_id in sent]
  test_masks.append(mask)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 4

# convert sang tensor
test_inputs = torch.tensor(test_ids)
# test_labels = torch.tensor(test_y)
test_masks = torch.tensor(test_masks)

# tạo data loader để train
test_data = TensorDataset(test_inputs, test_masks)#, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=BATCH_SIZE)

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

NUM_CLASSES = 5

# load file config
config_pr2_5 = RobertaConfig.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 3: CSR Sub Topic/CSR5/Bert Fine-tune/config.json',
    from_tf=False,
    num_labels=NUM_CLASSES,
    output_hidden_states=False
)

# load model
BERT_SA_pr2_5 = RobertaForSequenceClassification.from_pretrained(
    '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 3: CSR Sub Topic/CSR5/Bert Fine-tune/pytorch_model.bin',
    config = config_pr2_5
)

In [ ]:
BERT_SA_pr2_5.cuda()
print('Done')

Done


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Set the model to evaluation mode
BERT_SA_pr2_5.eval()

# Initialize empty lists to store predicted labels and true labels
predicted_labels = []
true_labels = []

# Predict the labels for the test data
for batch in tqdm(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        outputs = BERT_SA_pr2_5(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask
        )
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        # label_ids = b_labels.to('cpu').numpy()

        # Convert logits to class labels
        batch_predicted_labels = np.argmax(logits, axis=1)
        predicted_labels.extend(batch_predicted_labels)
        # true_labels.extend(label_ids)

# Attach the predicted labels to the test_df dataframe
# test_df_with_id['predicted_label_2'] = predicted_labels
test_df_pr2_5['predicted_label_3'] = predicted_labels

# Print the classification report
# print(classification_report(true_labels, predicted_labels))

# # Print the confusion matrix
# confusion_mat = confusion_matrix(true_labels, predicted_labels)
# print("Confusion Matrix:")
# print(confusion_mat)

  0%|          | 0/224 [00:00<?, ?it/s]

<ipython-input-153-1987e5340b03>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_pr2_5['predicted_label_3'] = predicted_labels


In [ ]:
test_df_pr2_5.head()

,ID,name,year,text,label,label_1,label_2,label_3,predicted_label_1,predicted_label_2,predicted_label_3
3,4,30. NGÂN HÀNG TMCP ĐẠI CHÚNG VIỆT NAM,2013,tình_thương có sức lan tỏa tự_nhiên vì_vậy khô...,CSR5_0,1,5,0,1,5,0
4,6,22. NGÂN HÀNG TMCP XĂNG DẦU PETROLIMEX,2011,đặc_biệt pg bank trực_tiếp tổ_chức nhiều chươn...,CSR5_0,1,5,0,1,5,0
6,10,1. NGÂN HÀNG TMCP VIỆT NAM THỊNH VƯỢNG,2016,vpbank tài_trợ trên tỷ đồng quỹ hội khuyến_học...,CSR5_4,1,5,4,1,5,3
9,16,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2019,tháng seabank phối_hợp cùng ủy_ban nhân_dân hu...,CSR5_0,1,5,0,1,5,0
10,17,27. NGÂN HÀNG TMCP CÔNG THƯƠNG VIỆT NAM,2016,vietinbank thực_hiện tài_trợ số tiền trên tỷ đ...,CSR5_0,1,5,0,1,5,0


### CSR3

In [ ]:
test_df_pr2_3['predicted_label_3'] = 2
test_df_pr2_3

In [ ]:
del BERT_SA_pr2_5
del BERT_SA_pr2_0
del BERT_SA_pr2_1


### CSR4

In [ ]:
test_x = test_df_pr2_4['text']

In [ ]:
# Đưa từng sentence qua tokenizer của PhoBERT để convert sang dạng token index với cùng chiều dài
# params
MAX_SEQ_LEN = 256 # chiều dài tối đa của một câu
# id của 1 số token đặc biệt
cls_id = 0  # đầu câu
eos_id = 2  # cuối câu
pad_id = 1  # padding

# Hàm xử lý dữ liệu trên từng sentence
def tokenize_line(line):
    tokenized = tokenizer.encode(line)

    l = len(tokenized)
    if l > MAX_SEQ_LEN: # nếu dài hơn thì cắt bỏ
        tokenized = tokenized[:MAX_SEQ_LEN]
        tokenized[-1] = eos_id # thêm EOS vào cuối câu
    else: # nếu ngắn hơn thì thêm padding vào
        tokenized = tokenized + [pad_id, ] * (MAX_SEQ_LEN - l)

    return tokenized

In [ ]:
tokenized_test_x = []

print('Tokenizing test set ... ')
for x in tqdm(test_x):
    tokenized_test_x.append(tokenize_line(x))

Tokenizing test set ... 


  0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
test_attention_mask = []
for x in tokenized_test_x:
    test_attention_mask.append(np.where(np.array(x) == 1, 0, 1))

In [ ]:
# Hàm extract feature của 1 line
def extract_line(tokenized, mask):
    tokenized = torch.tensor(tokenized).to(torch.long)
    mask = torch.tensor(mask)

    with torch.no_grad():
        last_hidden_states = phoBert(input_ids=tokenized, attention_mask=mask)

    feature = last_hidden_states[0][:, 0, :].numpy()

    return feature

In [ ]:
# Thực hiện tương tự trên tập test
test_size = len(tokenized_test_x)
EMBED_SIZE = 768
test_features = np.zeros(shape=(test_size, EMBED_SIZE))

start = time.time()

step = 20
for i in tqdm(range(0, test_size, step)):
    mask = test_attention_mask[i:i+step]
    tokenized = tokenized_test_x[i:i+step]
    feature = extract_line(tokenized, mask)
    test_features[i:i+step] = feature

# bert_test_features_path = '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 3: CSR Sub Topic/CSR4/Variable/bert_test_features.sav'
# pickle.dump(test_features, open(bert_test_features_path, 'wb'))

print('Finish extracting features after %d seconds' %(time.time() - start))
print('Output shape: ', test_features.shape)

  0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-161-b429c920d338>:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  mask = torch.tensor(mask)


Finish extracting features after 28 seconds
Output shape:  (38, 768)


In [ ]:
bert_logreg_path = '/content/drive/MyDrive/Research - CSR_NLP/Code/Problem 3: CSR Sub Topic/CSR4/Common Classifier Models/bert_logreg.sav'
logisticReg = pickle.load(open(bert_logreg_path, 'rb'))

In [ ]:
y_pred = logisticReg.predict(test_features)

In [ ]:
test_df_pr2_4['predicted_label_3'] = y_pred

<ipython-input-167-8587e87622ff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_pr2_4['predicted_label_3'] = y_pred


In [ ]:
test_df_pr2_4

# Attach test_df_pr2

In [ ]:
test_df_pr2 = pd.concat([test_df_pr2_0, test_df_pr2_1, test_df_pr2_2, test_df_pr2_3, test_df_pr2_4, test_df_pr2_5], axis=0)
test_df_pr2

In [ ]:
test_df_with_id

,ID,name,year,text,label,label_1,label_2,label_3,predicted_label_1
0,1,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2016,sacombank ban_hành bộ tiêu_chuẩn năng_lực quy_...,CSR0_8,1,0,8,1
1,2,5. NGÂN HÀNG TMCP SÀI GÒN THƯƠNG TÍN,2020,sacombank hướng sử_dụng nguyên_vật_liệu theo t...,CSR2_6,1,2,6,1
2,3,12. NGÂN HÀNG TMCP ĐÔNG NAM Á,2020,hệ_thống xử_lý nước_thải seabank đáp_ứng tiêu_...,CSR2_1,1,2,1,1
3,4,30. NGÂN HÀNG TMCP ĐẠI CHÚNG VIỆT NAM,2013,tình_thương có sức lan tỏa tự_nhiên vì_vậy khô...,CSR5_0,1,5,0,1
4,5,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2017,hiện_nay bộ chỉ_tiêu kpis phát_huy tốt vai_trò...,CSR0_10,1,0,10,0
...,...,...,...,...,...,...,...,...,...
3325,3326,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,trên thực_tế thành_viên hội_đồng_quản_trị thàn...,CSR0_1,1,0,1,1
3326,3327,28. NGÂN HÀNG TMCP NGOẠI THƯƠNG VIỆT NAM,2014,nhân_dịp chủ_tịch nước trương tấn sang tặng nh...,CSR5_3,1,5,3,1
3327,3328,2. NGÂN HÀNG TMCP KỸ THƯƠNG VIỆT NAM,2021,đồng_thời ông có kinh_nghiệm tham_gia quản_lý ...,No CSR,0,No CSR,No CSR,0
3328,3329,27. NGÂN HÀNG TMCP CÔNG THƯƠNG VIỆT NAM,2016,ngoài ra vietinbank đầu_tư hỗ_trợ xây_dựng khu...,CSR5_4,1,5,4,1


In [ ]:
merged_df = test_df_with_id.merge(test_df_pr2[['ID', 'predicted_label_2', 'predicted_label_3']], on='ID', how='left')
merged_df

In [ ]:
merged_df.to_csv('/content/drive/MyDrive/Research - CSR_NLP/final_model.csv')

In [ ]:
merged_df['predicted_label'] = merged_df.apply(lambda row: 'No CSR' if row['predicted_label_1'] == 0 else f'CSR{int(row["predicted_label_2"])}_{int(row["predicted_label_3"])}', axis=1)

In [ ]:
merged_df['predicted_label'] = merged_df['predicted_label'].replace('No_CSR', 'No CSR')

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3330 entries, 0 to 3329
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 3330 non-null   int64  
 1   name               3330 non-null   object 
 2   year               3330 non-null   int64  
 3   text               3330 non-null   object 
 4   label              3330 non-null   object 
 5   label_1            3330 non-null   int64  
 6   label_2            3330 non-null   object 
 7   label_3            3330 non-null   object 
 8   predicted_label_1  3330 non-null   int64  
 9   predicted_label_2  1844 non-null   float64
 10  predicted_label_3  1844 non-null   float64
 11  predicted_label    3330 non-null   object 
dtypes: float64(2), int64(4), object(6)
memory usage: 338.2+ KB


In [ ]:
from sklearn.metrics import confusion_matrix

# Extract the true labels and predicted labels
true_labels = merged_df['label']
predicted_labels = merged_df['predicted_label']

# Create the confusion matrix
confusion_mat = confusion_matrix(true_labels, predicted_labels)

print(confusion_mat)

[[   0    0    0 ...    0    0    3]
 [   0   85    1 ...    0    0   27]
 [   0    3   34 ...    0    0   14]
 ...
 [   0    0    0 ...  156    0    4]
 [   0    0    0 ...    4  101   12]
 [   0    4    1 ...    7    1 1066]]


In [ ]:
print(classification_report(true_labels, predicted_labels, zero_division=1, digits = 3))

              precision    recall  f1-score   support

      CSR0_0      1.000     0.000     0.000         4
      CSR0_1      0.825     0.733     0.776       116
     CSR0_10      0.895     0.607     0.723        56
     CSR0_11      0.818     0.360     0.500        25
      CSR0_2      0.826     0.487     0.613        39
      CSR0_3      0.659     0.659     0.659        41
      CSR0_4      0.917     0.579     0.710        38
      CSR0_5      0.774     0.805     0.789       128
      CSR0_6      1.000     0.111     0.200         9
      CSR0_7      1.000     0.333     0.500         6
      CSR0_8      0.870     0.755     0.808       212
      CSR0_9      0.918     0.589     0.718        95
      CSR1_0      1.000     0.400     0.571        25
      CSR1_1      0.861     0.408     0.554        76
      CSR1_2      0.857     0.735     0.791        49
      CSR1_3      1.000     0.000     0.000         4
      CSR1_4      1.000     0.000     0.000         1
      CSR2_0      0.643    